# Embedded Board Interface via Jupyter/Python Proof of Concept<a class="tocSkip">

&copy; 2020 by Michael Stanley (Mike.Stanley@ieee.org)<BR>
Last revised: 11 July 2020

# Introduction
This notebook uses a number of Python libraries to implement a simple interface for capturing sensor information
from an embedded MCU development board.    Graphics are provided using the bqplot library, and GUI widgets are created
using the standard ipywidgets library.
    
This notebook is intended both as proof of concept and as a teaching aide.  It is NOT intended for production use in collecting sensor data.  In that case, a non-Jupyter implemention makes more sense.  Please use the notebook in the spirit in which it is intended.
    
This draft of the Notebook is intended to work in conjunction with an NXP Semiconductor FRDM-STBC-AGM01/FRDM-K22F development board stackup.  Details of these boards can be found at https://www.nxp.com/freedom.  The K22F board should be programmed with a variant of the NXP sensor fusion library (https://www.nxp.com/sensorfusion.

![FRDM-K22F/FRDM-STBC-AGM01 Board Stackup](../images/agm01.jpg)    
# External code Imports
## Import from standard and 3rd party libraries

In [1]:
import ipywidgets as widgets        # Standard widget library for GUI construction 
from ipywidgets import interact, interactive, fixed, interact_manual
from pythreejs import pi, Preview   # Graphics library (used for 3D display)
import binascii                     # convert binary data to printable form
import threading                    # support for multithreading
import numpy as np                  # mathematical library
from pyquaternion import Quaternion # Quaternion utility library
from struct import *                # Functions used for unpacking binary data
import time                         # used for the sleep function
import math                         # standard math functions
#import yappi                       # you can optionally use yappi to profile parts of the code
import bqplot as bq                 # bqplot provides an object oriented plotting function with many capabilities
import collections                  # used to create dictionaries
from traitlets import link          # used for linking plot selection interaction functions
from IPython.display import clear_output, display  # used to implement GUI functions
from ipyfilechooser import FileChooser             # 3rd party code used to implement a file selection function
import os                           # primarily used to get the current working directory on the PC

## Import other modules of this application

In [2]:
from OutputWidgetHandler import *           # ipywidgets adaptation for use with standard python logging functions
from control import control, object_read    # objected oriented encapsulation of the serial interface to/from dev board
from Copter import createCopter, qconvert   # utility functions for creation of quad copter graphics

# Central Status Logging Facility

Because we have multiple GUI objects as well as multiple threads, a standard mechansim is required to send 
messages to the user. This is provided using the logger object.  Reference OutputWidgetHandler.py in this same 
directory for details of the implementation.

Example logger calls include:
*    logger.debug("Test Debug Message")
*    logger.info("Test Info Message")
*    logger.warning("Test Warning Message")
*    logger.error("Test Error Message")
*    logger.critical("Test Critical Message")

## Setup the logger

In [3]:
logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s - %(threadName)-10s %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)
#logger.setLevel(4)

# Set up the 3D model and Quaternion Functions
This application uses a predesigned 3D model of a quadcopter to represent orientation received from the development board, 
which is running a Kalman-filter based sensor fusion application.  The following lines of code call the object initializer,
but do not actually display the copter yet.  We'll get to that later.

Prerequisites: You must have "copter.stl" in the same directory as this notebook

In [4]:
copter = createCopter()
baseline_orientation = Quaternion(axis=[1, 0, 0], angle = 0)
q_temp = qconvert(baseline_orientation)
copter.quaternion=q_temp

def update_copter_orientation(Qnew):
    global copter, baseline_orientation
    # pythreejs uses xi,yj,zk,w format.  qconvert reorders from the (standard) pyquaternion q0 + q1*i + q2*j + q3*k form
    q_temp = qconvert(Qnew*baseline_orientation) # corresponds to 1st baseline_orientation followed by Qnew rotation
    copter.quaternion=q_temp

def update_copter_orientation_by_q_components(q0, q1, q2, q3):
    global copter
    # Quaternion is q0 + q1*i + q2*j + q3*k
    Qnew = Quaternion(q0, q1, q2, q3)
    update_copter_orientation(Qnew)

In [5]:
#display(Preview(copter))

Preview(child=Mesh(castShadow=True, geometry=BufferGeometry(attributes={'position': <BufferAttribute shape=(17…

# Variables for received packet data and parser jump table
The serial interface for this function is designed to be generic and re-useable.  Specific packet formats are handled by supplying the serial interface object with a jump table of functions for interpreting packet contents, and then storing away parsed data from same.

The packet_data variable must be initialized to a list wherein each cell will receive parsed data for a particular packet type.  The lenght of the list must match the number of packet types in the application.

Similarly, each cell in the jump table will contain a pointer to a function for decoding that particular packet type.  The actual functions are defined later in this notebook.

In [6]:
packet_data = [None]*14
jump_table = 14 * [None]

# Custom class for tracking serial traffic rates
The TimeKeeper class was designed to keep track of the number of packets sent over time and the instantaneous and average data rates over time.  The FRDM-K22F implementation does use a hard real-time clock to ensure data rates.  But it's always a good idea to check for dropped packets.  

In [7]:
import time
import threading
class TimeKeeper():
    def __init__(self):
        self.lock = threading.Lock()
        self.reset()
    def reset(self):
        with self.lock:
            self.current_rate=0
            self.average_rate=0
            self.packet_counter=0
            self.last_packet_counter=0
            self.last_time=time.time()
            self.start_time=self.last_time
    def incrementPacket(self):
        with self.lock:
            self.packet_counter += 1
    def snapshot(self):
        with self.lock:
            newTime = time.time()
            deltaTime = newTime - self.last_time
            self.last_time = newTime
            deltaPackets = self.packet_counter - self.last_packet_counter
            self.last_packet_counter = self.packet_counter
            if (deltaTime>0):
                self.current_rate = deltaPackets/deltaTime
            deltaTime = newTime - self.start_time
            if (deltaTime>0):
                self.average_rate = self.packet_counter/deltaTime
            return (self.packet_counter, self.current_rate, self.average_rate)

# Update GUI Function Called During Serial Streaming
Data streams to/from the sensor board and updates to the GUI are done in two separate background tasks.  In this way, GUI update rates can be separately throttled so that GUI updates (which are slow) do not starve the serial stream of CPU cycles.  This is a fairly standard way of managing tradeoffs in these types of application.
We keep a list of functions called "gui_updaters" which are periodically called to update various portions of the GUI.  This allows us to keep individual GUI update functions simple, but still executed in a controlled fashion.

In [8]:
gui_updaters = []     # List of functions to be called periodically to update the GUI
                      # This list will be added to in the code that follows  
stop_gui_updates = False  # Set to True to stop the GUI updates (normally done when the serial port is inactive)
skip_plot_updates = False  # Set to True to skip plot updates BUT NOT DISABLE serial port or GUI update thread
gui_update_interval = 0.5
def update_gui(serPort):
    global logger, gui_update_interval, stop_gui_updates, gui_updaters
    try:
        while not stop_gui_updates:
            with serPort.lock:
                for item in gui_updaters:
                    item()
            time.sleep(gui_update_interval)
        logger.info("Exiting update_gui() loop")
    except Exception as ex:
        logger.error("Exception in function update_gui(): ", str(ex))

# Set up the serial port interface
The serial port interface takes advantage of the FRDM-K22F UART over USB OpenSDA interface.  From the development board's perspective, we are communicating via a UART.  From the PC's perspective, it is a standard serial interface.  For this to work, you must have an OpenSDA debug interface on your board and appropriate drivers installed on your PC.  This notebook was developed with a FRDM-K22F equipped with a J-Link (Segger) debug interface.  Segger drivers can be downloaded from https://www.segger.com/downloads/jlink/.  You should also visit the [NXP Semiconductor page which details OpenSDA options for the FRDM-K22F](https://www.nxp.com/design/microcontrollers-developer-resources/ides-for-kinetis-mcus/opensda-serial-and-debug-adapter:OPENSDA?&tid=vanOpenSDA#FRDM-K22F).  On the embedded board side, you will need to be running a slightly modified version of the NXP 9-axis sensor fusion software.  That software will be published along with this notebook.  Please note that it should be fairly straightforward to apply the same code changes to any NXP development board running the NXP sensor fusion software.  That software is available as an option when configuring software development kits via the [MCUXpresso SDK web interface](https://www.nxp.com/design/software/development-software/mcuxpresso-software-and-tools/mcuxpresso-software-development-kit-sdk:MCUXpresso-SDK).

On the PC side, the serial interface is object oriented and defined in control.py.  This application assumes a single board/interface.  Public methods used here include:
* open()
* close()
* stop()
* print_stats()

The object_read() function is a thin wrapper that calls the class read() method.  It is run as a separate background thread.  See "Serial Port Enable/Disable" for details.  The class read_lock attribute is used to keep various threads from trying to use key resources at the same time.

In [9]:
serPort = control(logger, jump_table)  # This is the interface object
ports = control.get_port_names()       # This is a list of objects found attached to serial ports on this computer

# Define GUI Components
Technically speaking, you really don't need a graphical user interface to do data collection. But it helps tremendously to ensure that connections are properly made and that data received looks good in real-time, rather than waiting until a lengthly data collection session has been completed.

The GUI interface includes a "Main" tab that can be used to specify which port communicates with your board, along with fields to display recently received sensor values. Also included are tabs for displaying charts for each sensor type. The charts are "smart" and include the ability to connect/disconnect, turn data logging on/off, zoom in both X and Y dimensions and do separate dumps of sub-segments of the input waveforms.

## Serial Port Dropdown
You can determine what port your board is connected to by bringing up the Windows Device Manager Control Panel, expanding "Ports" and looking for one which includes "JLink CDC UART Port" in the description.  You should make sure that you select the same port using the sp_dropdown control defined here.  Note that port assignments tend to be somewhat static over time, so you could change "COM6" in the definition below to the port used by your board.  Then uncomment that line to have this selected as your default port.  The downside of doing this is that you will encounter an error in the future when you invariably get assigned to a different COM port.

In [10]:
sp_dropdown = widgets.Dropdown(
    options=ports,
    description='Ports:',
    disabled=False
    #,value="COM6",  # temporary hack to speed up development (actual quadcopter is always attached to this port)
) # note we are not displaying the widget yet

## Comms & Plot Control Functions
The serial port operates in a separate thread.  That thread is started/stopped via the toggle_serial_port() function callback to the toggle button that follows below.
### Button Callback Code

In [11]:
def stop_data():
    global stop_gui_updates, toggle1, serPort, logger
    # Disable serial port
    with serPort.read_lock: # lock prevents a race condition that can occur due to async stop & read functions
        stop_gui_updates = True;
        logger.debug("STOP REQUEST ACKNOWLEDGED")
        serPort.stop() # This should stop the currently executing thread
        serPort.close()
        toggle1.description="Enable Serial Port"
        logger.debug("Serial port has been closed")
        serPort.print_stats()

def toggle_serial_port(change):
    # Callback for serial port enable/disable button
    global logger, serPort, stop_gui_updates
    newState = change['new']
    if newState:
        # Enable serial port
        logger.debug("Button callback: Opening serial port")
        sts = serPort.open(sp_dropdown.value)
        if sts:
            logger.debug("Serial port successfully openeed")
            stop_gui_updates = False;
            toggle1.description="Disable Serial Port"
            logger.debug("Starting separate thread for serial port")
            t1 = threading.Thread(name="Control", target=object_read, kwargs=dict(object=serPort, debug=True, num_packets=0))
            t1.setDaemon(True)
            t1.start()
            logger.debug("Starting second thread for GUI updates")
            t2 = threading.Thread(name="GuiUpdate", target=update_gui, kwargs=dict(serPort=serPort))
            t2.setDaemon(True)
            t2.start()
            logger.debug("Serial port opened and secondary thread started.")            
        else:
            logger.error("ERROR!  Could not open ", sp_dropdown.value)
    else:
        stop_data()
    return()

### The Actual Serial Port Enable/Disable Button

In [12]:
toggle1=widgets.ToggleButton(
    value=False,
    description='Enable Serial Port',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button to start/stop serial data port logging',
    icon=''
) # note we are not displaying the widget yet

toggle1.observe(toggle_serial_port, 'value') # This ties in the callback f

## Plot freeze/unfreeze control
The plot freeze control can suspend plot updates without effecting data streaming operations.

In [13]:
def toggle_plot_freeze(change):
    # Callback for serial port enable/disable button
    global skip_plot_updates
    newState = change['new']
    if newState:
        skip_plot_updates = False
        toggle2.description="Freeze Plots"
    else:
        skip_plot_updates = True;
        toggle2.description="Re-enable Plots"
    return()

toggle2=widgets.ToggleButton(
    value=True,
    description='Freeze Plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click this button to freeze/start plots without affecting sensor communications',
    icon=''
) # note we are not displaying the widget yet

toggle2.observe(toggle_plot_freeze, 'value') # This ties in the callback f

## Data Logger
The act of recording sensor data to .csv files is encapsulated within the LoggingController class. Notice that this class incorporates both GUI and functional components of the data logger. It looks like an ipywidgets HBOX that contains all the functionality needed for specifying what output file to write and when to write it.

"Public" methods are the constructor, log() and loggerStartStop() functions. One of the GUI sub-components here is a file selection widget derived from the 3rd party ipyfilechooser Python library.

Data logs are in .csv format and include the following columns of data:

sample number
X-axis acceleration in g's
Y-axis acceleration in g's
Z-axis acceleration in g's
X-axis magnitometer reading in microTeslas
Y-axis magnitometer reading in microTeslas
Z-axis magnitometer reading in microTeslas
X-axis gyro reading in radians per second
Y-axis gyro reading in radians per second
Z-axis gyro reading in radians per second

In [14]:
logging_enabled = False  # Set to True to enable logging

class LoggingController(widgets.HBox):
    def __init__(self):
        self.lf = None
        self.logging_enabled = False;
        self.logIndex=0
        self.toggle=widgets.ToggleButton(
            value=False,
            description='Start Logging',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Click this button to turn sensor logging on and off',
            icon=''
        ) # note we are not displaying the widget yet
        
        # Create new FileChooser:
        # Path: current directory
        # File: log.csv
        # Title: <b>Logger Output File</b>
        # Show hidden files: no
        # Use the default path and filename as selection: yes
        self.file_dialog = FileChooser(
            os.getcwd(),
            filename='log.csv',
            title='<b>Logger Output File</b>',
            show_hidden=False,
            select_default=True
        )
        self.toggle.observe(self.loggerStartStop, 'value') # This ties in the callback f
        
        super().__init__(
            children = [self.toggle, self.file_dialog]
        )
    def logXYZ(self, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ):
        if not self.lf==None:
            print('{0}, {1:6.3f}, {2:6.3f}, {3:6.3f}, {4:6.3f}, {5:6.3f}, {6:6.3f}, {7:6.3f}, {8:6.3f}, {9:6.3f}'\
                .format(self.logIndex, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ), file=self.lf)
            self.logIndex += 1
            
    def loggerStartStop(self, change):
        # Callback for logging start/stop button
        newState = change['new']
        if newState:
            try:
                self.lf = open(self.file_dialog.selected, 'wt')
                print('Index, accelX, accelY, accelZ, magX, magY, magZ, gyroX, gyroY, gyroZ', file=self.lf)
                self.logging_enabled = True
                self.toggle.description="Stop Logging"
            except Exception as ex:
                logger.error("Could not open logger output file.  Check to see if it is open elsewhere. ", str(ex))
        else:
            if not self.lf==None:
                self.lf.close()
                self.lf = None
                self.logIndex = 0
            self.logging_enabled=False
            self.toggle.description="Start Logging"
        return()

loggingController = LoggingController()

## Button and Callback for Clearing Console
Clearing past messages from the console is trivially implemented with this button and callback.

In [15]:
def on_button1_clicked(b):
    handler.clear()  

button1 = widgets.Button(
    description='Clear Console',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon=''
)
button1.on_click(on_button1_clicked)

## Accel/Mag/Gyro/Quaternion Values Table
The "Main" tab of the GUI contains a table that displays recently received sensor values for the accelerometer, magnetometer and gyroscope.  That table is defined using an ipywidgets GridspecLayout function.

### Table Definition

In [16]:
from ipywidgets import GridspecLayout
colNames = ['X/q0', 'Y/q1', 'Z/q2', 'VM/q3']
rowNames = ['Accel', 'Mag', 'Gyro', 'quaternion']
def table(colNames, rowNamees):
    grid = GridspecLayout(5, 5, grid_gap='0px', width='100%')
    rowNums = range(1,len(rowNames)+1)
    colNums = range(1,len(colNames)+1)
    for i in rowNums:
        for j in colNums:
            grid[i, j] = widgets.FloatText(value=0.0, layout=widgets.Layout(width='110px', height='auto'))
    for idx, colName in zip(colNums, ['X/q0', 'Y/q1', 'Z/q2', 'VM/q3']):
        grid[0, idx] = widgets.Label(value=colName, layout=widgets.Layout(width='110px', height='auto'))
    for idx, rowName in zip(rowNums, ["Accel (g's)", "Mag (uT's)", "Gyro (dps)", 'quaternion']):
        grid[idx, 0] = widgets.Label(value=rowName, layout=widgets.Layout(width='110px', height='auto'))
    grid[0,0] = widgets.Text(value="0 / 0.0", layout=widgets.Layout(width='110px', height='auto'))
    return(grid)
sensor_grid=table(colNames, rowNames)
#display(sensor_grid)  # Uncomment for debug

### Table Update Functions

In [17]:
def update_sensor_grid_triaxial_row(row, x,y,z):
    global sensor_grid
    sensor_grid[row,1].value=x
    sensor_grid[row,2].value=y
    sensor_grid[row,3].value=z
    sensor_grid[row,4].value=math.sqrt(x*x+y*y+z*z)
def update_accel_row(x,y,z):
    update_sensor_grid_triaxial_row(1,x,y,z)
def update_mag_row(x,y,z):
    update_sensor_grid_triaxial_row(2,x,y,z)
def update_gyro_row(x,y,z):
    update_sensor_grid_triaxial_row(3,x,y,z)
def update_quat_row(q0, q1, q2, q3):
    global sensor_grid
    sensor_grid[4,1].value=q0
    sensor_grid[4,2].value=q1
    sensor_grid[4,3].value=q2
    sensor_grid[4,4].value=q3
update_counter=TimeKeeper()
def update_counter_cell():
    global update_counter
    (packet_count, current_rate, average_rate) = update_counter.snapshot()
    s = '%d / %6.3f' % (packet_count, current_rate)
    sensor_grid[0,0].value=s
def increment_counter():
    global update_counter
    logger.debug("updating counter")
    update_counter.incrementPacket()
gui_updaters.append(update_counter_cell)

## Assemble the contents of what will become the main tab
The graphical user interface utilizes a tabbed interface.  Individual tabs contents are defined and then assembled below.  We have a central logging console immediately below the tabbed interface, so that it is visible regardless of which tab is selected.  This console is written to using the standard Python logging library.

In [18]:
q_layout = widgets.Layout(width='110px')
style = {'description_width': 'initial'}
box_layout = widgets.Layout(align_items='center')
main_right_row1 = widgets.HBox([sp_dropdown, toggle1, button1])
main_right_row2 = widgets.HBox([sensor_grid])
main_right_panel = widgets.VBox([main_right_row1, main_right_row2], layout=box_layout)
main = widgets.HBox([Preview(copter), main_right_panel])

## Plot Implementation
### Circular Buffer
New data points are written directly from the serial port to very efficient circular buffers.  These are designed so that the buffers do not need to be copied for every new sample coming in.  Only when the plot is updated are those more time intensive tasks done.

A circular buffer class is defined to facilitate this. 

The implication of the above is that the length of the plots is fixed in time to a specific window into the past.  This keeps the plotting functions operating efficiently.  If you want the full history of data, you should use the data logger to write data to the file system as it arrives, and then post-process the resulting .csv data after the logging session is complete.

In [19]:
class plot_buffer():
    def __init__(self, size):
        self.b = np.zeros(size, dtype=np.float32)
        self.next = 0
        self.size=size
        self.full = False;
    def push(self, value):
        self.b[self.next]=value
        self.next += 1
        if self.next>=self.size:
            self.full = True
            self.next = 0
    def unwrapped(self):
        if (not self.full):
            return(self.b)
        else:
            return(np.concatenate((self.b[self.next:], self.b[0:self.next-1])))    

### Tri-Axial Plot Implementation
The xyz_plot class is designed to plot (X, Y & Z) plus the vector magnitude.  It includes a number of GUI controls plus the actual plots.  We use the bqplot library because it offers some very nice interactive plot features built right into the plot library itself.  These include pan/zoom and brush selection, plus the ability to update the plot points in time.

This implementation plots values versus SAMPLE INDEX into the past.  This is not exactly the same as plotting versus timestamps.  However, because the FRDM-K22F DOES use a hard real-time for control of sampling intervals, it is functionally the same.   Using sample indices for the X-axis is a simplification that is necessary in order for us to use circular input buffers, which in turn is necessary to keep the plotting functions fast enough to keep up with the input data streams.

In [20]:
class xyz_plot():
    global logger
    def plotCallback(self, change):
        pass
    
    def __init__(self, maxLen, title, xAxisLabel, yAxisLabel):
        self.title = title
        self.lock = threading.Lock()
        self.ts = bq.LinearScale()
        self.xs = bq.LinearScale()
        self.ys = bq.LinearScale()
        self.zs = bq.LinearScale()
        self.vms = bq.LinearScale()
        self.maxLen = maxLen
        self.xData  = plot_buffer(maxLen)
        self.yData  = plot_buffer(maxLen)
        self.zData  = plot_buffer(maxLen)
        self.vmData = plot_buffer(maxLen)
        self.tData = np.linspace(-maxLen,0,maxLen)       
        self.xLine = bq.Lines(x=self.tData, y=[self.xData.b], scales={'x':self.ts, 'y':self.ys}, labels=['X'], colors=['red'], display_legend=True)        
        self.yLine = bq.Lines(x=self.tData, y=[self.yData.b], scales={'x':self.ts, 'y':self.ys}, labels=['Y'], colors=['blue'], display_legend=True)        
        self.zLine = bq.Lines(x=self.tData, y=[self.zData.b], scales={'x':self.ts, 'y':self.ys}, labels=['Z'], colors=['black'], display_legend=True)
        lw = 0.7
        self.xLine.stroke_width=lw      
        self.yLine.stroke_width=lw
        self.zLine.stroke_width=lw  
        def_tt = bq.Tooltip(fields=['x', 'y'], formats=['2.3f', '2.2f'], labels=['Time', 'VM'])
        self.vmLine = bq.Lines(x=self.tData, y=[self.vmData.b],  tooltip=def_tt, unhovered_style={'opacity': 0.5}, \
            scales={'x':self.ts, 'y':self.ys}, labels=['VM'], display_legend=True)
        self.vmLine.colors=['DarkOrange']
        self.vmLine.fill = 'bottom'
        self.vmLine.fill_opacities = [0.10]
        self.vmLine.stroke_width=1
        self.vmLine.marker = 'circle'
        
        self.ax_x = bq.Axis(scale=self.ts, label=xAxisLabel)
        self.ax_y = bq.Axis(scale=self.ys, orientation='vertical', label=yAxisLabel)
        self.figure = bq.Figure(marks=[self.xLine, self.yLine, self.zLine, self.vmLine], \
                                axes=[self.ax_x, self.ax_y], title=title)
        
        panzoom = bq.PanZoom(scales={'y': [self.ys]})
        intervalSelector = bq.interacts.BrushIntervalSelector(scale=self.ts, marks=[self.xLine, self.yLine, self.zLine, self.vmLine])
        intervalSelector.observe(self.plotCallback, names=['selected'])
        
        from collections import OrderedDict
        selection_interacts = widgets.ToggleButtons(options=OrderedDict([('PanZoom', panzoom), \
                    ('BrushIntervalSelector', intervalSelector), ('VM Tooltips', None)]))
        link((selection_interacts, 'value'), (self.figure, 'interaction'))
        self.figure.layout.height = '400px'
        self.figure.layout.width = '70%'
        self.visibility_controls = interactive(self.set_visible, x=True, y=True, z=True, vm=False)

        plotButton = widgets.Button(
            description='Download Plot',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Will dump current plot into a file on your computer',
            icon=''
        )
        plotButton.on_click(self.on_plotButton_clicked)

        self.widget = widgets.HBox([self.figure, widgets.VBox( \
                [widgets.Label("Visibility Controls"), self.visibility_controls, \
                widgets.Label("Plot Interactions"), selection_interacts, \
                widgets.Label("Other Functions"), plotButton, \
                ])])
        
    def on_plotButton_clicked(self, b):
        fn = self.title+'.png'
        self.figure.save_png(fn)
    
    def set_visible(self, x,y,z,vm):
        self.xLine.visible = x
        self.xLine.display_legend = x
        self.yLine.visible = y
        self.yLine.display_legend = y
        self.zLine.visible = z
        self.zLine.display_legend = z
        self.vmLine.visible = vm
        self.vmLine.display_legend = vm
    def push(self,x,y,z):
        with self.lock:
            self.xData.push(x)
            self.yData.push(y)
            self.zData.push(z)
            self.vmData.push(np.sqrt(x*x + y*y + z*z))
    def update(self):
        with self.lock:
            x = self.xData.unwrapped()
            self.xLine.y = []
            self.xLine.y = x
            
            y = self.yData.unwrapped()
            self.yLine.y = []
            self.yLine.y = y
            
            z = self.zData.unwrapped()
            self.zLine.y = []
            self.zLine.y = z
            
            vm = self.vmData.unwrapped()
            self.vmLine.y = []
            self.vmLine.y = vm

numPlotPoints = 256
accelPlot = xyz_plot(numPlotPoints, 'Acceleration', 'Time (sample intervals)', 'Acceleration (g\'s)')
magPlot = xyz_plot(numPlotPoints, 'Magnetic_Field', 'Time (sample intervals)', 'Magnetic Field (uT\'s)')
gyroPlot = xyz_plot(numPlotPoints, 'Angular_Rotation', 'Time (sample intervals)', 'Rotation (dps)')

def update_plot():
    global accelPlot, magPlot, gyroPlot, skip_plot_updates
    if not skip_plot_updates:
        accelPlot.update()
        magPlot.update()
        gyroPlot.update()
    return

## Tabbed Interface
Here's where we pull together the various functions into the tabbed interface.  The "gui" variable represents the top level of the graphical hierarchy.

In [21]:
tab_titles = ['Main', 'Accelerometer', 'Magnetometer', 'Gyroscope']
tab = widgets.Tab()
for i in range(len(tab_titles)):
    tab.set_title(i, tab_titles[i])
commonPlotControls = widgets.VBox([widgets.HBox([toggle1, toggle2]), loggingController])
tab.children = [main, \
               widgets.VBox([accelPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([magPlot.widget, commonPlotControls], layout={'border':'1px solid black'}), \
               widgets.VBox([gyroPlot.widget, commonPlotControls], layout={'border':'1px solid black'})]
layout = {
    'width': '100%',
    'border': '1px solid black',
}
gui = widgets.VBox([tab, handler.out], layout=layout)

# Utility functions
These are a few general purpose utility functions used elsewhere in the code.

In [22]:
# from: https://www.geeksforgeeks.org/python-program-to-convert-a-tuple-to-a-string/
# Python3 code to convert tuple  
# into string 
def convertTuple(tup): 
    str =  ''.join(tup) 
    return str

def check_expected_float(e, a, tol):
    global logger
    if (a<(e-tol)) or (a>(e+tol)):
        logger.error("Expected %s = %d, got %d", e, a, t)
        return False;
    else:
        return True;

def check_expected_int(e, a, t):
    global logger
    if e!=a:
        logger.error("Expected %s = %f, got %f", e, a, t)
        return False;
    else:
        return True;

# Serial port and GUI interface code

The embedded implementation sends two different packet types for consumption by this logger. The first is identical to the standard packet type 1 found in the NXP Sensor Fusion Library.  The second packet type (type 14) is a custom self-test designed to validate the embedded board to python interface.

## Process packet type 1

In [23]:
packet1_template = Struct('<13h2B')
def read_packet1(packet):
    global logger
    #logger.log(5, "Starting read_packet1")
    #logger.log(5, "packet to be processed = %s", binascii.hexlify(bytearray(packet)))
    try:
        quat_sf = 30000
        global packet_data
        packet_data[0] = packet1_template.unpack_from(packet, 6)
        update_sensors()
        increment_counter()  # update our packet counter
    except:
        pl = len(packet)
        logger.error("Exception in function read_packet1(%s).  Packet length=%d", binascii.hexlify(bytearray(packet) ), pl)
jump_table[0] = read_packet1

## Process packet type 14 (pack/unpack self-test)

In [24]:
packet14_template = Struct('<6B3hH2lL3f')
packet_14_test_complete = False # only need to run this once

def read_packet14(packet):
    global logger, serPort, handler, packet_14_test_complete, update_counter, update_counter_display
    if packet_14_test_complete:
        return()
    packet_14_test_complete = True
    #logger.info("Starting read_packet14")
    #logger.info("packet to be processed = %s", binascii.hexlify(bytearray(packet)))
    try:
        packet_data[13] = packet14_template.unpack_from(packet, 0)
    except:
        logger.error("Packet14 unpack_from failed")
        stop_data()
    try:
        (pt, pn, b1, b2, b3, b4, s1, s2, s3, us, L1, L2, UL, f1, f2, f3) = packet_data[13]
    except:
        logger.error("Packet14 named tuple unpack failed")
        stop_data()
    try:
        sts = \
        check_expected_int(0x0E, pt, "byte") and \
        check_expected_int(0xAA, b1, "byte") and \
        check_expected_int(0x55, b2, "byte") and \
        check_expected_int(0x0f, b3, "byte") and \
        check_expected_int(0xf0, b4, "byte") and \
        check_expected_int(1, s1, "short") and \
        check_expected_int(-1, s2, "short") and \
        check_expected_int(256, s3, "short") and \
        check_expected_int(62960, us, "unsigned short") and \
        check_expected_int(1, L1, "long") and \
        check_expected_int(-1, L2, "long") and \
        check_expected_int(305419896, UL, "unsigned long") and \
        check_expected_float(1, f1, .00001) and \
        check_expected_float(-1, f2, .00001) and \
        check_expected_float(-1.123456, f3, .00001)
        if sts:
            logger.info("Packet14 check completed OK")
        else:
            logger.error("Packet14 check failed.  Please inspect preceeding messages")
    except:
        pl = len(packet)
        logger.error("Exception in function read_packet14(%s).  Packet length=%d", binascii.hexlify(bytearray(packet) ), pl)
    logger.log(5,"Completing read_packet14")
jump_table[13] = read_packet14

## Function for updating GUI sensors values, plots, etc.

The update_sensors() packet is called by the read_packet1() function shown above.  It's job is to push new data to both plot and data logger consumers.

The update_sensors_display() is called periodically (in a separate thread from the serial communications) to update the graphic displays.

In [25]:
def update_sensors():
    global packet_data, logger, q0_echo, q1_echo, q2_echo, q3_echo, q_mag
    #logger.log(5, "entering update_copter")
    if packet_data[0] == None:
        pass
    else:
        quat_sf = 30000
        accel_sf = .00012207 # 122.07 ug  per LSB
        mag_sf = 0.1         #   0.   uT  per LSB
        gyro_sf = 0.05       #   0.05 dps per LSB
        (accX, accY, accZ, magX, magY, magZ, gyroX, gyroY, gyroZ, q0, q1, q2, q3, flags, boardIDs) = packet_data[0]
        accX = accel_sf * accX
        accY = accel_sf * accY
        accZ = accel_sf * accZ
        magX = mag_sf * magX
        mayY = mag_sf * magY
        magZ = mag_sf * magZ
        gyroX = gyro_sf * gyroX
        gyroY = gyro_sf * gyroY
        gyroZ = gyro_sf * gyroZ
        accelPlot.push(accX, accY, accZ)
        magPlot.push(magX, magY, magZ)
        gyroPlot.push(gyroX, gyroY, gyroZ)
        loggingController.logXYZ(accX, accY, accZ, magX, magY, magZ, gyroX, gyroY, gyroZ)

        
def update_sensors_display():
    global packet_data, logger, q0_echo, q1_echo, q2_echo, q3_echo, q_mag, accelPlot
    #logger.log(5, "entering update_copter")
    if packet_data[0] == None:
        pass
    else:
        quat_sf = 30000
        accel_sf = .00012207 # 122.07 ug  per LSB
        mag_sf = 0.1         #   0.   uT  per LSB
        gyro_sf = 0.05       #   0.05 dps per LSB
        (accX, accY, accZ, magX, magY, magZ, gyroX, gyroY, gyroZ, q0, q1, q2, q3, flags, boardIDs) = packet_data[0]
        q0 = q0/quat_sf
        q1 = q1/quat_sf
        q2 = q2/quat_sf
        q3 = q3/quat_sf
        accX = accel_sf * accX
        accY = accel_sf * accY
        accZ = accel_sf * accZ
        magX = mag_sf * magX
        mayY = mag_sf * magY
        magZ = mag_sf * magZ
        gyroX = gyro_sf * gyroX
        gyroY = gyro_sf * gyroY
        gyroZ = gyro_sf * gyroZ
        update_copter_orientation_by_q_components(q0, q1, q2, q3)
        update_accel_row(accX, accY, accZ)
        update_mag_row(magX, magY, magZ)
        update_gyro_row(gyroX, gyroY, gyroZ)
        update_quat_row(q0, q1, q2, q3)

gui_updaters.append(update_sensors_display)
gui_updaters.append(update_plot)


# Display the GUI
And finally, let's display the final application. 

In [26]:
display(gui)